In [1]:
import h2o
h2o.init()
from h2o.estimators.word2vec import H2OWord2vecEstimator
import nltk
# nltk.download('stopwords') -- might need if running nltk + stopwords for the first time
from nltk.corpus import stopwords
from h2o.automl import H2OAutoML
import pandas as pd
from sklearn import metrics

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,6 days 23 hours 21 mins
H2O cluster timezone:,Europe/Vienna
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.5
H2O cluster version age:,"14 days, 14 hours and 41 minutes"
H2O cluster name:,H2O_from_python_mackenzie_dm3nzl
H2O cluster total nodes:,1
H2O cluster free memory:,1.219 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [25]:
train_data = h2o.upload_file("/home/mackenzie/Downloads/EnglishCleanedTrainingData.csv") # check if path correct 
test_data = h2o.upload_file("/home/mackenzie/Downloads/EnglishCleanedTestingData.csv") # check if path correct
STOP_WORDS = set(stopwords.words('english'))
new_stopwords = ['rt', 'co', 'http', 'u', 'got', 'get']
STOP_WORDS = STOP_WORDS.union(new_stopwords)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [4]:
print("Break train tweets into sequence of words")
train_words = tokenize(train_data["tweet"])

Break train tweets into sequence of words


In [5]:
print("Break test tweets into sequence of words")
test_words = tokenize(test_data["tweet"])

Break test tweets into sequence of words


In [6]:
print("Build word2vec model for train")
w2v_model_one = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
w2v_model_one.train(training_frame=train_words)

Build word2vec model for train
word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [7]:
print("Build word2vec model for test")
w2v_model_two = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
w2v_model_two.train(training_frame=test_words)

Build word2vec model for test
word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [8]:
# could do synonym check here
#print("Sanity check - find synonyms for the word 'teacher'")
#w2v_model.find_synonyms("teacher", count = 5)

In [9]:
print("Calculate a vector for each train tweet")
tweet_vecs_train = w2v_model_one.transform(train_words, aggregate_method = "AVERAGE")
print(tweet_vecs_train)

Calculate a vector for each train tweet


C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
0.0223424,-0.08276,0.0387922,0.117605,0.10378,-0.144005,-0.0221281,-0.051491,-0.0144988,0.00602809,0.222964,0.114629,-0.128073,-0.103262,-0.184455,-0.0345309,-0.0431599,0.0863207,-0.202299,0.224225,0.0215012,0.215449,-0.122652,0.0115363,0.208421,0.151796,-0.157925,-0.0497596,0.165338,-0.0447987,-0.0355493,0.197404,-0.302941,0.175013,0.00974624,-0.0929491,-0.165572,-0.047391,-0.228681,-0.0234547,-0.222493,-0.0358672,0.156457,-0.121174,-0.239643,-0.092991,-0.127105,0.0259335,0.100321,-0.0313455,-0.113834,-0.194439,0.183633,0.0233283,0.251628,0.0351301,0.0904468,-0.0372926,-0.0700557,-0.0526358,0.0645934,-0.144038,0.0168826,0.194091,-0.0205084,0.0335193,-0.123215,-0.184751,0.170322,0.0740542,-0.0467131,-0.176256,-0.0881352,-0.0574672,-0.0337711,-0.124141,-0.0452782,0.0344,0.0148767,0.253006,-0.0315287,-0.254408,-0.0812914,0.0343174,0.0772073,-0.0617911,-0.182489,-0.220473,0.0101707,0.0424757,0.14839,-0.0269312,-0.0207631,0.105923,-0.0374845,0.035177,0.0222166,0.194937,0.0802423,0.0933859
0.0381571,0.169677,-0.0946149,0.0558397,0.0266164,-0.0336636,0.0751325,-0.157822,-0.0662286,-0.0330098,0.147064,0.139625,-0.0766034,0.0753417,-0.0813428,0.0705755,-0.179138,0.0661921,-0.158312,0.10535,0.108928,0.264944,0.0136087,0.121737,0.152939,0.147995,0.0068351,-0.00768577,0.179074,0.0671987,-0.00392963,0.0722276,-0.210974,-0.124551,0.0498734,-0.10903,-0.162239,0.0283855,-0.327184,0.0283496,-0.199846,-0.170235,0.0960226,-0.054669,-0.192298,-0.169993,-0.278767,0.0990923,0.0206655,0.0559498,0.0124686,-0.217594,0.0711856,-0.123334,0.146967,-0.123879,-0.0663025,0.0562728,0.0676319,-0.0215916,0.0829631,0.11396,-0.0103352,-0.0999446,0.0175469,0.179792,-0.188006,0.0814401,0.289559,0.136967,-0.0663057,-0.141992,-0.319676,-0.0111743,-0.1079,-0.210902,0.0981346,-0.0665335,0.124551,-0.0121625,-0.00209863,-0.207908,-0.185202,0.0137445,0.107493,-0.0245979,-0.19279,-0.120658,0.00970476,0.1241,0.136172,-0.149274,-0.0558557,0.0765942,0.113389,0.15758,0.118811,0.0921255,0.0523585,0.123106
0.156697,0.18083,-0.0624462,0.0231771,0.0430882,-0.075262,0.00655932,0.0488314,-0.0172671,-0.0445129,0.0983548,0.073057,-0.198447,0.0500282,0.0139142,0.168126,-0.120425,0.0204765,-0.142426,0.0602858,0.122715,0.120716,-0.0905239,0.0472035,0.0387592,0.0989807,-0.116968,-0.170702,0.1045,-0.0339078,-0.0549988,0.0542825,-0.131366,-0.0726431,0.0546257,0.0186057,0.0132361,0.0231228,-0.247582,-0.00924552,-0.227218,-0.0764011,0.0529758,-0.0698664,-0.233807,-0.0769302,-0.201596,0.124064,0.0219946,0.00283156,-0.0465681,-0.351317,0.237269,-0.0779144,0.182555,-0.0879206,-0.0499739,0.0883217,-0.0686838,-0.032174,-0.035887,-0.0704311,-0.00101519,-0.00792553,-0.0839945,0.0255661,-0.131935,-0.0329768,0.135967,0.166322,-0.0279789,-0.0849111,-0.129975,-0.0596101,-0.0739678,-0.247904,0.139237,0.0206789,0.0757857,0.0879412,0.0201232,-0.169221,-0.0679391,0.027705,0.00329124,-0.0867621,-0.178825,-0.264275,-0.004378,0.131204,0.115545,-0.099328,-0.0649829,0.0216034,0.0935499,0.0574648,-0.00661883,0.131036,0.0792279,0.115624
-0.108762,0.224019,0.0491488,-0.0458032,-0.0307439,-0.0665347,-0.0226533,-0.00578452,-0.0755648,0.00195552,0.165161,0.0633906,-0.0841122,0.156898,-0.256173,0.0894957,-0.142387,0.100941,-0.161361,0.0173705,0.093694,0.104144,-0.184319,0.210381,0.0375314,0.0796243,0.0315992,-0.0655946,0.162442,-0.0260141,-0.130015,-0.129768,-0.108081,0.0668278,-0.0368491,-0.0999875,-0.0755118,0.177272,-0.195265,0.0371625,-0.153785,-0.00489053,0.0484189,-0.122989,-0.0417165,-0.0763641,-0.168227,0.20354,0.134843,-0.117026,-0.102748,-0.0650835,0.115502,0.00540144,0.0646049,-0.107289,0.0367208,0.0256136,-0.0240284,-0.1

In [10]:
print("Calculate a vector for each test tweet")
tweet_vecs_test = w2v_model_two.transform(test_words, aggregate_method = "AVERAGE")

Calculate a vector for each test tweet


In [11]:
# see lines 10 to 12 in the demo for word2Vec for using ml on the vecs and such
#print(tweet_vecs)

In [12]:
# preparing the train and test data sets
# now convert tweet vecs and labels to a pandas dataframe and back to h2o dataframe
# data = job_titles[valid_job_titles,:].cbind(job_title_vecs[valid_job_titles,:])
train = tweet_vecs_train.cbind(train_data["labels"])
test = tweet_vecs_test.cbind(test_data["labels"])


In [13]:
# more on data prep
x = train.columns         # x: A list/vector of predictor column names or indexes. 
                          # This argument only needs to be specified if the user wants to exclude columns from the 
                          # set of predictors. If all columns (other than the response) should be used in prediction, 
                          # then this does not need to be set.

y = "labels"              # This argument is the name (or index) of the response column
x.remove(y)

# need to set train and test
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [14]:
# now the AUTO-ML piece comes in
aml = H2OAutoML(max_models=10) #max_models=10 or 20?, seed?
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [15]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20190703_143453,0.379212,0.352589,0.3191,0.101825
StackedEnsemble_AllModels_AutoML_20190703_143453,0.380945,0.351513,0.31898,0.101748
GLM_grid_1_AutoML_20190703_143453_model_1,0.384846,0.347966,0.319396,0.102014
GBM_1_AutoML_20190703_143453,0.38608,0.343009,0.318445,0.101407
GBM_2_AutoML_20190703_143453,0.3924,0.343937,0.318668,0.101549
XGBoost_1_AutoML_20190703_143453,0.394923,0.338059,0.316211,0.0999894
GBM_3_AutoML_20190703_143453,0.395216,0.347392,0.319843,0.102299
XGBoost_2_AutoML_20190703_143453,0.397681,0.344188,0.321341,0.10326
GBM_4_AutoML_20190703_143453,0.401665,0.352246,0.319949,0.102368
XGBoost_3_AutoML_20190703_143453,0.410509,0.35616,0.328232,0.107736


In [16]:
# The leader model is stored here
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20190703_143453
No model summary for this model


ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.04370837949664802
RMSE: 0.20906549092724036

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.10182483318575941
RMSE: 0.31910003632992495


In [17]:
preds = aml.predict(test)
print(preds)

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict,p0,p1,p2
2,0.107316,0.170745,0.721938
2,0.0450257,0.0502032,0.904771
2,0.16064,0.226383,0.612977
2,0.0877273,0.122501,0.789772
2,0.0917272,0.131997,0.776276
2,0.22616,0.116045,0.657795
2,0.0590925,0.0577924,0.883115
2,0.0954635,0.126629,0.777908
2,0.107068,0.218627,0.674305
2,0.0656908,0.0806016,0.853708


In [18]:
var = preds["predict"].cbind(test[y])
print(var)

predict,labels
2,1
2,1
2,2
2,1
2,1
2,1
2,1
2,1
2,1
2,0


In [21]:
# convert to pandas dataframe
y_test = h2o.as_list(test[y], use_pandas=True)
y_pred = h2o.as_list(preds["predict"])
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))

[[   9    9  268]
 [  28   51 3759]
 [   4   64  765]]
0.16643130925963284
